<a href="https://colab.research.google.com/github/alexlopespereira/mba_enap/blob/main/CD/Notebooks/Aula8_ContarDosesCovid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Contar doses de vacinas da COVID-19

In [1]:
import pandas as pd
from google.colab import auth
auth.authenticate_user()
print('Authenticated')
## Defina o id do seu projeto no bigquery!!!!!
project_id = 'hardy-messenger-229417' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!



Authenticated


### Dica: Restrinja seus testes aos estados do AC, AP e RR para diminuir o tamanho da sua Query.

In [2]:


df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,vacina
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única|^Dose$') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89') and sigla_uf in ('AC','AP','RR')
group by sigla_uf, vacina;
''', project_id=project_id)

df_vacina.head()

/tmp/ipython-input-4248334805.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''


,sigla_uf,vacina,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,AC,86,185531,100467,84938,109,17,0
1,AC,87,457364,243218,156853,49567,7726,0
2,AC,85,422293,222991,195394,3584,324,0
3,AC,89,25492,12102,13375,15,0,0
4,AC,88,14251,0,0,1874,150,12227


### Calcular a semana epidemiológica (domingo a sábado) a partir da data de vacinação

In [3]:
df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,vacina
    ,(case
        when vacina='86' then 'Coronavac'
        when vacina='87' then 'Pfizer'
        when vacina='88' then 'Janssen'
        when vacina in ('85','89') then 'Astrazeneca'
        else 'Outro'
      end
    ) as vacina_apelido
    ,DATE_TRUNC(data_aplicacao, WEEK(SUNDAY)) as semana
    ,DATE_TRUNC(data_aplicacao, MONTH) as mes
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única|^Dose$') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89')
group by sigla_uf, vacina, semana, mes
''', project_id=project_id)

df_vacina.head()


/tmp/ipython-input-2997802042.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina = pd.io.gbq.read_gbq('''


,sigla_uf,vacina,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,AC,85,Astrazeneca,2021-08-29,2021-08-01,4663,243,4420,0,0,0
1,AC,85,Astrazeneca,2021-10-03,2021-10-01,7029,424,6586,16,3,0
2,AC,85,Astrazeneca,2021-11-28,2021-11-01,771,91,549,130,1,0
3,AM,85,Astrazeneca,2021-05-30,2021-06-01,71833,48005,23828,0,0,0
4,AM,85,Astrazeneca,2021-08-01,2021-08-01,36879,9662,27214,3,0,0


In [4]:
df_vacina.to_gbq("enapdatasets.vacinacao",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )

/tmp/ipython-input-2614492827.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df_vacina.to_gbq("enapdatasets.vacinacao",
100%|██████████| 1/1 [00:00<00:00, 7667.83it/s]


## Query para projeção de 2a dose da vacina do Covid
### Requisito de negócio: 2a dose deve ser aplicada a partir de 56 dias após a 1a dose.

In [ ]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.mes, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enap-331414.enapdatasets.vacinacao` v
JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 56 DAY) as semana_proj
    FROM `enap-331414.enapdatasets.vacinacao`
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
order by v.sigla_uf, v.vacina, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
''', project_id=project_id)

df_vacina_proj.head()

,sigla_uf,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
0,AC,Astrazeneca,2021-03-14 00:00:00+00:00,2021-03-01 00:00:00+00:00,554,549,5,0,0,0,10,2021-03-14 00:00:00+00:00,AC,Astrazeneca
1,AC,Astrazeneca,2021-03-21 00:00:00+00:00,2021-03-01 00:00:00+00:00,6734,6712,22,0,0,0,792,2021-03-21 00:00:00+00:00,AC,Astrazeneca
2,AC,Astrazeneca,2021-03-28 00:00:00+00:00,2021-04-01 00:00:00+00:00,681,680,1,0,0,0,1747,2021-03-28 00:00:00+00:00,AC,Astrazeneca
3,AC,Astrazeneca,2021-03-28 00:00:00+00:00,2021-04-01 00:00:00+00:00,681,680,1,0,0,0,3,2021-03-28 00:00:00+00:00,AC,Astrazeneca
4,AC,Astrazeneca,2021-03-28 00:00:00+00:00,2021-04-01 00:00:00+00:00,681,680,1,0,0,0,1,2021-03-28 00:00:00+00:00,AC,Astrazeneca


In [ ]:
# df_vacina_proj['semana_proj'].max()
df_vacina_proj.iloc[20:60]

,sigla_uf,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
20,AC,Astrazeneca,2021-05-02 00:00:00+00:00,2021-05-01 00:00:00+00:00,9125,7589,1536,0,0,0,1,2021-05-02 00:00:00+00:00,AC,Astrazeneca
21,AC,Astrazeneca,2021-05-09 00:00:00+00:00,2021-05-01 00:00:00+00:00,12542,11494,1048,0,0,0,549,2021-05-09 00:00:00+00:00,AC,Astrazeneca
22,AC,Astrazeneca,2021-05-09 00:00:00+00:00,2021-05-01 00:00:00+00:00,12542,11494,1048,0,0,0,1,2021-05-09 00:00:00+00:00,AC,Astrazeneca
23,AC,Astrazeneca,2021-05-16 00:00:00+00:00,2021-05-01 00:00:00+00:00,10148,9414,734,0,0,0,6712,2021-05-16 00:00:00+00:00,AC,Astrazeneca
24,AC,Astrazeneca,2021-05-16 00:00:00+00:00,2021-05-01 00:00:00+00:00,10148,9414,734,0,0,0,1,2021-05-16 00:00:00+00:00,AC,Astrazeneca
25,AC,Astrazeneca,2021-05-23 00:00:00+00:00,2021-05-01 00:00:00+00:00,9795,5675,4120,0,0,0,680,2021-05-23 00:00:00+00:00,AC,Astrazeneca
26,AC,Astrazeneca,2021-05-23 00:00:00+00:00,2021-05-01 00:00:00+00:00,9795,5675,4120,0,0,0,360,2021-05-23 00:00:00+00:00,AC,Astrazeneca
27,AC,Astrazeneca,2021-05-23 00:00:00+00:00,2021-05-01 00:00:00+00:00,9795,5675,4120,0,0,0,4749,2021-05-23 00:00:00+00:00,AC,Astrazeneca
28,AC,Astrazeneca,2021-05-23 00:00:00+00:00,2021-05-01 00:00:00+00:00,9795,5675,4120,0,0,0,210,2021-05-23 00:00:00+00:00,AC,Astrazeneca
29,AC,Astrazeneca,2021-05-30 00:00:00+00:00,2021-06-01 00:00:00+00:00,5714,4107,1607,0,0,0,3421,2021-05-30 00:00:00+00:00,AC,Astrazeneca


### Query com parâmetro



```
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.mes, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enap-331414.enapdatasets.vacinacao` v
JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enap-331414.enapdatasets.vacinacao`
    ) as vp
ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
order by v.sigla_uf, v.vacina, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj
```



### Corrigindo anos menores do que 2021

In [ ]:
df_vacina = pd.io.gbq.read_gbq('''
SELECT
    sigla_uf
    ,vacina
    ,(case
        when vacina='86' then 'Coronavac'
        when vacina='87' then 'Pfizer'
        when vacina='88' then 'Janssen'
        else 'Astrazeneca'
      end
    ) as vacina_apelido
    ,(case when EXTRACT(YEAR FROM data_aplicacao) >= 2021 then DATE_TRUNC(data_aplicacao, WEEK(SUNDAY)) else PARSE_DATE('%d/%m/%Y',  '03/01/2021') end) as semana
    ,DATE_TRUNC(data_aplicacao, MONTH) as mes
    ,count(*) as qt_total
    ,sum(case when regexp_contains(dose,'1ª Dose$|^Dose$|Inicial') then 1 else 0 end) as qt_D1
    ,sum(case when regexp_contains(dose,'2ª Dose$') then 1 else 0 end) as qt_D2
    ,sum(case when regexp_contains(dose,'Reforço') then 1 else 0 end) as qt_Reforco
    ,sum(case when regexp_contains(dose,'Dose Adicional|3ª Dose') then 1 else 0 end) as qt_Adicional
    ,sum(case when regexp_contains(dose,'Única') then 1 else 0 end) as qt_Unica
FROM `basedosdados.br_ms_vacinacao_covid19.microdados_vacinacao`
where vacina in ('85','86','87','88','89')
group by sigla_uf, vacina, semana, mes
''', project_id=project_id)


df_vacina.to_gbq("enapdatasets.vacinacao",
  project_id=project_id,
  chunksize=40000,
  if_exists='replace',
  )
df_vacina.head()


1it [00:04,  4.31s/it]


,sigla_uf,vacina,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica
0,MS,85,Astrazeneca,2021-08-22,2021-08-01,46451,182,46269,0,0,0
1,RO,85,Astrazeneca,2021-07-25,2021-07-01,47228,28063,19165,0,0,0
2,RO,85,Astrazeneca,2021-05-02,2021-05-01,23157,16791,6366,0,0,0
3,DF,85,Astrazeneca,2021-04-18,2021-04-01,38761,35249,3512,0,0,0
4,DF,85,Astrazeneca,2021-05-02,2021-05-01,70975,51711,19264,0,0,0


## Query de Projeção com FULL OUTER JOIN (BigQuery)

In [6]:
df_vacina_proj = pd.io.gbq.read_gbq('''
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.mes, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enapdatasets.vacinacao` v
FULL OUTER JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL 56 DAY) as semana_proj
    FROM `enapdatasets.vacinacao`
    ) as vp ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
order by v.sigla_uf, v.vacina, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj

''', project_id=project_id)

df_vacina_proj.head()

/tmp/ipython-input-244296362.py:1: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_vacina_proj = pd.io.gbq.read_gbq('''


,sigla_uf,vacina_apelido,semana,mes,qt_total,qt_D1,qt_D2,qt_Reforco,qt_Adicional,qt_Unica,qt_D2_Proj,semana_proj,sigla_uf_proj,vacina_apelido_proj
0,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,0,1900-02-18,RO,Astrazeneca
1,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1900-02-18,RO,Coronavac
2,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1900-10-14,BA,Coronavac
3,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1901-02-17,BA,Pfizer
4,None,None,NaT,NaT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1,1921-06-05,SC,Astrazeneca


### Código com a quanitdade de dias parametrizado
```
SELECT v.sigla_uf, v.vacina_apelido, v.semana, v.mes, v.qt_total, v.qt_D1, v.qt_D2, v.qt_Reforco, v.qt_Adicional, v.qt_Unica, vp.qt_D2_Proj, vp.semana_proj, vp.sigla_uf_proj, vp. vacina_apelido_proj
FROM `enapdatasets.vacinacao` v
FULL OUTER JOIN (
    SELECT sigla_uf as sigla_uf_proj, vacina_apelido as vacina_apelido_proj, qt_D1 as qt_D2_Proj, DATE_ADD(semana, INTERVAL @qtd_dias_proj_d2 DAY) as semana_proj
    FROM `enapdatasets.vacinacao`
    ) as vp ON v.sigla_uf=vp.sigla_uf_proj and v.vacina_apelido=vp.vacina_apelido_proj and vp.semana_proj=v.semana
order by v.sigla_uf, v.vacina, v.semana, vp.semana_proj, vp.sigla_uf_proj, vp.vacina_apelido_proj

```

### Daria para calcular o conjunto de todas as semanas no python/pandas?